# Curvature: types of curvature


## Gauss and mean curvature
The Weingartentensor $\nabla_{\mathcal{S}} \nu\in \mathbb{R}^{3\times 3}$ is symmetric and has always zero as one eigenvalue with $\nu$ the corresponding eigenvector as from $\|\nu\|=1$ we get $(\nabla_{\mathcal{S}} \nu)\nu=0$. The other two eigenvalues $\kappa_1$ and $\kappa_2$ are the two principal curvatures of the surface and the Gauss and mean curvatures are given by the product and mean of them, or equivalently as the determinant and trace of the Weingartentensor (interpreted as $2\times2$ matrix)
\begin{align}
K_g &= \kappa_1\kappa_2 = \det(\nabla_{\mathcal{S}} \nu)\\
K_m &= 0.5(\kappa_1+\kappa_2)=0.5\,\mathrm{tr}(\nabla_{\mathcal{S}} \nu)
\end{align}

Let's investigate the curvatures of different types of surfaces.

In [ ]:
from ngsolve import *
from netgen.meshing import MeshingStep
from netgen.csg import *
from netgen import meshing
from ngsolve.webgui import Draw
#import netgen.gui

order=4

def ComputeCurvature(mesh, ev=False):
    nsurf = specialcf.normal(3)

    mean_curvature = 0.5*Trace(Grad(nsurf))
    gauss_curvature = Cof(Grad(nsurf))*nsurf*nsurf

    kappa1 = mean_curvature+sqrt(mean_curvature**2-gauss_curvature)
    kappa2 = mean_curvature-sqrt(mean_curvature**2-gauss_curvature)

    print("Gauss curvature")
    Draw(gauss_curvature, mesh,"gauss")
    print("Mean curvature")
    Draw(mean_curvature, mesh, "mean")
    print("first principal curvature")
    Draw(kappa1, mesh, "kappa1")
    print("second principal curvature")
    Draw(kappa2, mesh, "kappa2")

    if ev:
        eigs = Grad(nsurf).Eig()
        ev1 = CF( (eigs[0], eigs[1], eigs[2]) )
        ev2 = CF( (eigs[3], eigs[4], eigs[5]) )
        ev3 = CF( (eigs[6], eigs[7], eigs[8]) )
        v1 = IfPos(Norm(eigs[9]-kappa1)-1e-3,IfPos(Norm(eigs[10]-kappa1)-1e-3,ev3,ev2),ev1)
        v2 = IfPos(Norm(eigs[9]-kappa2)-1e-3,IfPos(Norm(eigs[10]-kappa2)-1e-3,ev3,ev2),ev1)

        Draw(v1, mesh, "v1")
        Draw(v2, mesh, "v2")
    return

For the sphere the principal curvatures are reciproce to the radius $r$ and thus
\begin{align}
\kappa_i = \frac{1}{r},\qquad K_g=\frac{1}{r^2},\qquad K_m=\frac{1}{r}.
\end{align}
The sphere is called an elliptic surface as the Gauss curvature is positive.

In [ ]:
geo = CSGeometry()
geo.Add(Sphere(Pnt(0,0,0),3))
mesh = Mesh(geo.GenerateMesh(maxh=1, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(order)

ComputeCurvature(mesh)

For the cylinder one of the principal curvatures is zero and the other reciproce to the radius $r$ and thus
\begin{align}
\kappa_1 = \frac{1}{r},\quad \kappa_2=0,\qquad K_g=0,\qquad K_m=\frac{1}{2r}.
\end{align}
The cylinder is of parabolic type as the Gauss curvature is zero.

In [ ]:
geo       = CSGeometry()
cyl       = Cylinder(Pnt(0,0,0), Pnt(1,0,0), 2)
right     = Plane( Pnt(3,0,0), Vec(1,0,0))
left      = Plane(Pnt(0,0,0), Vec(-1,0,0))
finitecyl = cyl * left * right
geo.AddSurface(cyl, finitecyl)
mesh = Mesh(geo.GenerateMesh(maxh=1)).Curve(order)

ComputeCurvature(mesh)

An hyperbolic structure has negative Gauss curvature as one principal curvature is negative and the other positive.

In [ ]:
R = 1

mapping = lambda x,y,z : ( R*cos(pi*x-pi/2)*cosh(2*y-1), \
                          R*sinh(2*y-1), R*sin(-pi*x+pi/2)*cosh(2*y-1) )        
mesh = Mesh(meshing.SurfaceGeometry(mapping).GenerateMesh(quads=False,nx=8,ny=8)).Curve(order)
        
ComputeCurvature(mesh, ev=False)

Pseudosphere (Tractricoid) $K=-1$, $H=0.5(\sinh(u)-\mathrm{csch}(u))$

In [ ]:
gu = lambda u : log(tan(u/2))+cos(u)
mapping = lambda x,y,z : ( sin(pi/9/2+x*pi/9)*cos(y*2*pi), sin(pi/9/2+x*pi/9)*sin(y*2*pi), gu(pi/9/2+x*pi/9) )
mesh = Mesh(meshing.SurfaceGeometry(mapping).GenerateMesh(quads=False,nx=6,ny=8)).Curve(order)
        
ComputeCurvature(mesh, ev=False)

Katenoid: $\kappa_1=-\frac{1}{\cosh^2u}$, $\kappa_2=\frac{1}{\cosh^2u}$, $H=0$, $K<0$

In [ ]:
mapping = lambda x,y,z : ( cosh(2*(x-0.5))*cos(y*2*pi), cosh(2*(x-0.5))*sin(y*2*pi), 2*(x-0.5) )
mesh = Mesh(meshing.SurfaceGeometry(mapping).GenerateMesh(quads=False,nx=6,ny=6)).Curve(order)
        
ComputeCurvature(mesh, ev=False)